In [129]:
# When input a lattice basis S, this program returns a new basis for the same basis.
# This includes SampleD Al. as a lattice vector sampling oracle.  
# The condition of s is: s > ||Gram-schmitch(S)||.\omega(\sqrt(log(n)))
import numpy as np
import random

In [130]:
#============================================
def tn(x):
    return log(x,2)
def gaussian(s,c,x):
    return exp((-pi*(x-c)^2)/s^2)
#===========================================
def sampleZalgorithm(s, c):
    """
    On input tuple (n,s,c), this Al. returns an integer number with Gaussian distribution
    *Input: 
    n: Security parameter
    s: Gaussian parameter
    c: Gaussian mean
    *Output:
    x: A random number with Gaussian distribution 
    """
    l = int(c -s*tn(n))
    r = int(c+ s*tn(n))
    i=0
    while i<1:
        x= random.randint(l,r)
        accept = random.uniform(0,1)
        if accept < gaussian(s,c,x): 
            samples = x
            i=i+1
            break
    return samples

#============================================
def sampleDalgorithm(B,s,c): #B is a list of m vectors
    """
    * Input:
    B: A basis of any lattice lambda (as a Matrix whose column vectors form the whole lattice)
    s: Gaussian parameter
    c: Gaussian mean vector 
    * Output:
    A vector in lattice lambda with gaussian distribution
    """
    m=B.rank()
    B=B.T
    v=zero_vector(ZZ,m)
    G=(B.gram_schmidt())[0]
    Bc = list(zero_vector(ZZ,m))+[c]
    Bv = list(zero_vector(ZZ,m))+[v]
    for i in range(m,0,-1):
        ci = Bc[i].dot_product(G[i-1])/(G[i-1].dot_product(G[i-1]))
        si= s/((G[i-1].dot_product(G[i-1]))^(1/2))
        zi= sampleZalgorithm(si,ci)
        Bc[i-1]=Bc[i]-zi*B[i-1]
        Bv[i-1] = Bv[i] +zi*B[i-1]
    return(Bv[0])

#==============================================
def ToBasis(V,S):  #V, S are 2 matrices
    """
    *Input:
    V: Independent short lattice vectors
    S: A basis of lattice L
    *Output: 
    A new short basis S_1 of lattice L
    """
    Q=matrix(ZZ,(S**(-1))*V)
    T = Q.echelon_form()
    U= Q*(T**(-1))
    S_1=matrix(ZZ,S*U)
    return(S_1)

#=================================================
    #generate lattice and it's basis
def Random_Matrix(m):
    """
    *Input: An integer m 
    *Output: A random RANK m squared matrix B with coefficients from 0 to q
    The output matrix has all column vectors independent
    """
    B=(matrix(ZZ,np.random.randint(q,size=(m,1)))) #first vector of B
    size_B=1
    while size_B < m:
        A = (matrix(ZZ,np.random.randint(q,size=(m,1))))
        h=(B.T).rank()
        if ((B.augment(A)).T).rank()==h+1:
            B=B.augment(A) 
            size_B=size_B+1
    return(B)

#===================================================
def RandBasis(S, s):
    """
    *Input: 
    S: Basis of a lattice L (as a matrix form)
    s: Gaussian parameter   (note that s > ||gram_schmitd(S)||*omega(sqrt(log(n,2))))
    *Output:
    A new short basis of lattice L
    """
    #preparation for SampleD
    m=S.ncols()
    c=zero_vector(ZZ,m)
    V=matrix(sampleDalgorithm(S,s,c))
    i=1
    while i <m:
        if (V.stack(matrix(sampleDalgorithm(S,s,c)))).rank()>i:
            V=V.stack(matrix(sampleDalgorithm(S,s,c)))
            i=i+1
    V=V.T
    A=ToBasis(V,((S.T).hermite_form()).T)
    return A

#===================================================
# A TEST CASE
q= next_prime(2^10)
    #def GenTrap(n,q,r):
n=next_prime(2^10)
m= 20 #int(2*n*log(q,2))

#l= int(log(q,r))
#d=int(2*n*log(q,2))
S=(Random_Matrix(m).T)

In [81]:
# Normalise parameter s
def gram_schmidt_norm(S):
    m=S.rank()
    S_gram=((S.T).gram_schmidt()[0])
    S_gram_norm = int(max(((S_gram.rows())[i]).norm() for i in range(m)))+1
    return S_gram_norm
gram_schmidt_norm(S)

11

In [61]:
s= int((S_gram_norm)*sqrt(log(n,2)))+1;s


4696

In [62]:
S_1=RandBasis(S, s)

In [59]:
#Check if S_1 is a basis of lattice
print(S.det()/(S_1.det()))

1


In [126]:
S= random_matrix(ZZ,3,3);S


[ 0 -1 -1]
[-1  1 28]
[-1  3  0]

In [127]:
np.linalg.qr(S)

(array([[ 0.        , -0.57735027,  0.81649658],
        [ 0.70710678, -0.57735027, -0.40824829],
        [ 0.70710678,  0.57735027,  0.40824829]]),
 array([[ -1.41421356,   2.82842712,  19.79898987],
        [  0.        ,   1.73205081, -15.58845727],
        [  0.        ,   0.        , -12.24744871]]))

In [128]:
S.gram_schmidt()

(
[       0       -1       -1]  [      1       0       0]
[      -1    -27/2     27/2]  [  -29/2       1       0]
[-810/731   30/731  -30/731], [   -3/2 -79/731       1]
)

In [ ]:
A=random_matrix(ZZ,259,259)